In [76]:
import psycopg2
import configparser
import requests
import json
import os
from datetime import datetime, timedelta

def get_workspaces(api_token):
    headers = {
        "Content-Type": "application/json",
    }
    response = requests.get(
        'https://api.track.toggl.com/api/v9/workspaces',
        headers=headers,
        auth=(api_token, 'api_token')
    )
    response.raise_for_status()
    return response.json()

def get_detailed_report(api_token, workspace_id, since_date, until_date):
    report_data = []
    headers = {
        "Content-Type": "application/json",
    }
    page = 1
    while True:
        response = requests.get(
            f'https://api.track.toggl.com/reports/api/v2/details?workspace_id={workspace_id}&since={since_date}&until={until_date}&user_agent=api_test&page={page}',
            headers=headers,
            auth=(api_token, 'api_token')
        )
        response.raise_for_status()
        data = response.json()
        report_data.extend(data['data'])

        print(f'Successfully fetched page {page}')
        page += 1

        if not data['data']:
            break
    return report_data

def process_data(data, conn):
    cur = conn.cursor()
    for item in data:
        processed_item = {
            "id" : item.get("id", ""),
            "user_name": item.get("user", ""),
            "Client": item.get("client", ""),
            "Project": item.get("project", ""),
            "Description": item.get("description", ""),
            "Billable": item.get("is_billable", ""),
            "Start_date": item["start"].split('T')[0] if "start" in item else "",
            "Start_time": item["start"].split('T')[1].split('+')[0] if "start" in item else "",
            "End_date": item["end"].split('T')[0] if "end" in item else "",
            "End_time": item["end"].split('T')[1].split('+')[0] if "end" in item else "",
            "Duration_ms": item.get("dur", ""),
            "Tags": item["tags"] if "tags" in item and item["tags"] else []
        }
        columns = list(processed_item.keys())
        values = [processed_item[column] for column in columns]
        query = f'INSERT INTO test.toggl_history ({",".join(columns[0:])}) VALUES (%s{",%s"*(len(values)-1)}) ON CONFLICT (id) DO UPDATE SET {",".join([f"{column}=excluded.{column}" for column in columns[1:]])}'
        cur.execute(query, values)
    conn.commit()

def main():
    config = configparser.ConfigParser()
    config.read('config.ini')
    api_token = config.get('Toggl', 'api_token')
    mode = config.get('Download', 'mode')

    if mode == 'init':
        since_date = config.get('Payload', 'since')
        until_date = config.get('Payload', 'until')
    else:  # mode is 'recurring'
        days_back = config.getint('Download', 'recurring_days_back')
        until_date = datetime.today()
        since_date = (until_date - timedelta(days=days_back)).strftime('%Y-%m-%d')
        until_date = until_date.strftime('%Y-%m-%d')


    host = config.get('PostgreSQL', 'host')
    dbname = config.get('PostgreSQL', 'dbname')
    user = config.get('PostgreSQL', 'user')
    password = config.get('PostgreSQL', 'password')

    conn = psycopg2.connect(
        host=host,
        dbname=dbname,
        user=user,
        password=password
    )

    workspaces = get_workspaces(api_token)
    for workspace in workspaces:
        workspace_id = workspace['id']
        report_data = get_detailed_report(api_token, workspace_id, since_date, until_date)
        process_data(report_data, conn)
        print(f'Inserted data for workspace {workspace_id} from {start_date} to {end_date}')

    conn.close()

if __name__ == "__main__":
    main()

Successfully fetched page 1
Successfully fetched page 2
Successfully fetched page 3
Successfully fetched page 4
Successfully fetched page 5
Successfully fetched page 6
Successfully fetched page 7
Successfully fetched page 8
Successfully fetched page 9
Successfully fetched page 10
Successfully fetched page 11
Successfully fetched page 12
Successfully fetched page 13
Successfully fetched page 14
Successfully fetched page 15
Successfully fetched page 16
Successfully fetched page 17
Successfully fetched page 18
Successfully fetched page 19
Successfully fetched page 20
Successfully fetched page 21
Successfully fetched page 22
Successfully fetched page 23
Successfully fetched page 24
Successfully fetched page 25
Successfully fetched page 26
Successfully fetched page 27
Successfully fetched page 28
Successfully fetched page 29
Successfully fetched page 30
Successfully fetched page 31
Successfully fetched page 32
Successfully fetched page 33
Successfully fetched page 34
Successfully fetched pa

In [9]:
import psycopg2
import pandas as pd
import configparser
# Connect to the PostgreSQL database
config = configparser.ConfigParser()
config.read('config.ini')

host = config.get('PostgreSQL', 'host')
dbname = config.get('PostgreSQL', 'dbname')
user = config.get('PostgreSQL', 'user')
password = config.get('PostgreSQL', 'password')

conn = psycopg2.connect(
    host=host,
    dbname=dbname,
    user=user,
    password=password
)

# Execute a SELECT statement to fetch the 100 newest rows
cur = conn.cursor()
cur.execute("SELECT * FROM test.toggl_history ORDER BY id DESC LIMIT 100")
rows = cur.fetchall()

# Create a dataframe from the fetched rows
df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
# Close the database connection
conn.close()
# Print the dataframe
df.head(10)



,id,user_name,client,project,description,billable,start_date,start_time,end_date,end_time,duration_ms,tags
0,3140161463,Leo,AXA - Westhouse | DBI Analytics,AXA | Kosten VU,KVU Feedback,True,2023-09-25,16:00:00,2023-09-25,17:00:00,3600000,[Dataset]
1,3140074488,Alex Below,Internal - DBI | DBI Analytics,DBI | Others internal,Toggle - Databricks internes Projekt,False,2023-09-25,16:10:00,2023-09-25,16:49:12,2352000,[]
2,3140047233,Charlotte Jordan,AXA - Westhouse | DBI Analytics,AXA | Projektmanagement,Abstimmung Leo,True,2023-09-25,16:06:15,2023-09-25,16:12:48,393000,[Align (internal)]
3,3140024557,Charlotte Jordan,AXA - Westhouse | DBI Analytics,AXA | Projektmanagement,Tickets ableiten Axa,True,2023-09-25,15:56:58,2023-09-25,16:05:58,540000,[Project Management]
4,3140010622,Charlotte Jordan,Internal - DBI | DBI Analytics,DBI | Others internal,daily stand up,False,2023-09-25,09:30:00,2023-09-25,10:00:00,1800000,[Align (internal)]
5,3139777395,Charlotte Jordan,Senec - RightPeopleGroup | DBI Analytics,SENEC | Projektmanagement,Aufwandsdokumentation Axa Q4,True,2023-09-25,13:54:23,2023-09-25,15:49:47,6924000,[Project Management]
6,3139765971,Charlotte Jordan,Senec - RightPeopleGroup | DBI Analytics,SENEC | PBI Vertrieb,Quoten Produktgruppen,True,2023-09-25,13:47:05,2023-09-25,13:54:07,422000,[Dataset]
7,3139759647,Alex Below,Internal - DBI | DBI Analytics,DBI | Others internal,Toggle - Databricks internes Projekt,False,2023-09-25,13:43:04,2023-09-25,15:56:10,7986000,[Develop new features]
8,3139757539,Alex Below,Senec - RightPeopleGroup | DBI Analytics,SENEC | PBI Vertrieb,Unterstützung bei Measures im Vertriebs-Report,True,2023-09-25,13:35:00,2023-09-25,13:43:03,483000,[]
9,3139735123,Alex Below,Internal - DBI | DBI Analytics,DBI | Others internal,Daily,False,2023-09-25,09:30:00,2023-09-25,09:45:00,900000,[Meeting]


In [70]:
import requests
from base64 import b64encode
from datetime import datetime, timedelta

import configparser

# Read the config file
config = configparser.ConfigParser()
config.read('config.ini')

# Get the Toggl API credentials from the config file
email = config.get("Toggl", "email")
password = config.get("Toggl", "password")
workspace_id = 1345262

# Calculate the start and end dates for the 1-year timeframe
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")

# Build the request payload
payload = {
    "start_date": start_date,
    "end_date": end_date
}

# Make the API request
url = f"https://api.track.toggl.com/reports/api/v3/workspace/{workspace_id}/search/time_entries"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Basic " + b64encode(f"{email}:{password}".encode("ascii")).decode("ascii")
}
response = requests.post(url, json=payload, headers=headers)

# Print the response JSON
print(response.json())


[{'user_id': 8604864, 'username': 'Jan Busse', 'project_id': 175752833, 'task_id': None, 'billable': False, 'description': 'Meeting', 'tag_ids': [11643898], 'billable_amount_in_cents': None, 'hourly_rate_in_cents': None, 'currency': 'USD', 'time_entries': [{'id': 2662915431, 'seconds': 744, 'start': '2022-09-26T17:45:35+02:00', 'stop': '2022-09-26T17:57:59+02:00', 'at': '2022-11-01T23:28:44+00:00'}], 'row_number': 1}, {'user_id': 2864925, 'username': 'Christopher Matt', 'project_id': 175752833, 'task_id': None, 'billable': False, 'description': 'Orga: Abstimmung World AI Summit', 'tag_ids': [12775805], 'billable_amount_in_cents': None, 'hourly_rate_in_cents': None, 'currency': 'USD', 'time_entries': [{'id': 2663027494, 'seconds': 680, 'start': '2022-09-26T18:39:59+02:00', 'stop': '2022-09-26T18:51:19+02:00', 'at': '2022-11-01T23:30:13+00:00'}], 'row_number': 2}, {'user_id': 2864925, 'username': 'Christopher Matt', 'project_id': 175752833, 'task_id': None, 'billable': False, 'descriptio